## STACKING MODELS WITH MLXtend

In [2]:
#import libraries
import pandas as pd
import numpy as np

from sklearn.linear_model import RidgeCV, LinearRegression
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, StackingRegressor

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error as MSE, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, FunctionTransformer 
from sklearn.compose import ColumnTransformer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression, VarianceThreshold
from sklearn.impute import SimpleImputer


In [3]:
df = pd.read_csv("cleaned_df.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df.shape

(1460, 81)

In [5]:
null_values = df.isnull().sum().sort_values(ascending = False)
print(null_values[null_values > 0])

MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
GarageFinish      81
GarageType        81
Electrical         1
dtype: int64


In [6]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

## Feature Engineering and Preprocessing Pipeline

#### Automatic feature creator

Define a class to create features automatically whenever there is a new X input.

In [9]:
# class for feature creation
class FeatureCreator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None): # temporarily create features to be used in column transformation
        if y is not None:
            df = X.copy()
            df['SalePrice'] = y
            df['price_sqft'] = df['SalePrice'] / df['GrLivArea'].replace(0, np.nan)

            self.neighborhood_avg_price_sqft = df.groupby('Neighborhood')['price_sqft'].mean().to_dict()
            self.nbhood_avghouse_price = df.groupby("Neighborhood")["SalePrice"].mean().to_dict()
            self.SubClass_avg_price_sqft = df.groupby("MSSubClass")["price_sqft"].mean().to_dict()
            self.Zoning_avg_price_sqft = df.groupby("MSZoning")["price_sqft"].mean().to_dict()
            self.yearbuilt_avg_price = df.groupby("YearBuilt")["SalePrice"].mean().to_dict()
            df['age'] = df['YrSold'] - df['YearBuilt']
            df['age_afterRemodel'] = df['YrSold'] - df['YearRemodAdd']
            self.age_avg_price = df.groupby("age")["SalePrice"].mean().to_dict()
            self.age_afterRemodel_price_sqft = df.groupby("age_afterRemodel")["SalePrice"].mean().to_dict()
            self.rooms_avg_price = df.groupby("TotRmsAbvGrd")["SalePrice"].mean().to_dict()
            self.overalcond_avg_price = df.groupby("OverallCond")["SalePrice"].mean().to_dict()
        return self

    def transform(self, X):
        df = X.copy()

        # Basic features
        df['total_area'] = df.get('GrLivArea', 0) + df.get('TotalBsmtSF', 0) + df.get('GarageArea', 0)
        df['total_bathrooms'] = (
            df.get('FullBath', 0) +
            0.5 * df.get('HalfBath', 0) +
            df.get('BsmtFullBath', 0) +
            0.5 * df.get('BsmtHalfBath', 0)
        )

        # Age features
        df['age'] = df.get('YrSold', 0) - df.get('YearBuilt', 0)
        df['age_afterRemodel'] = df.get('YrSold', 0) - df.get('YearRemodAdd', 0)

        # Mapping price new features — without overwriting originals
        df['nbd_avg_price_sqf'] = df['Neighborhood'].map(self.neighborhood_avg_price_sqft)
        df['nbd_avg_price_sqf'] = df['nbd_avg_price_sqf'].fillna(np.mean(list(self.neighborhood_avg_price_sqft.values())))

        df['nbd_avg_house_price'] = df['Neighborhood'].map(self.nbhood_avghouse_price)
        df['SubClass_avg_price_sqf'] = df['MSSubClass'].map(self.SubClass_avg_price_sqft)
        df['Zoning_avg_price_sqf'] = df['MSZoning'].map(self.Zoning_avg_price_sqft)
        df['yearbuilt_avg_price'] = df['YearBuilt'].map(self.yearbuilt_avg_price)
        df['age_avg_price'] = df['age'].map(self.age_avg_price)
        df['age_afterRemodel_price'] = df['age_afterRemodel'].map(self.age_afterRemodel_price_sqft)
        df['rooms_avg_price'] = df['TotRmsAbvGrd'].map(self.rooms_avg_price)
        df['overalcond_avg_price'] = df['OverallCond'].map(self.overalcond_avg_price)

        return df


### Split the dataset

In [11]:
df = df.dropna(subset=['SalePrice'])

# Split features and target
X = df.drop(columns = ['SalePrice'])
X = X.dropna(axis = 'columns')

y = df['SalePrice']  # KEEP AS IS (raw)

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature creation from training data for the sole purpose of selecting numerical features for columtransfomer
feature_creator = FeatureCreator()
X_train_created = feature_creator.fit_transform(X_train, y_train)

#  Use only the transformed X_train to define feature types
num_features = X_train_created.select_dtypes(include='number').columns.tolist()
cat_features = X_train_created.select_dtypes(include=['object', 'category']).columns.tolist()


#### Select polynomial features to be added

In [13]:
#Apply PolynomialFeatures only to a few important columns 

Selected_poly_features= ['LotFrontage', 'yearbuilt_avg_price', 'OverallQual', 'GrLivArea', 
                         'total_area', 'nbd_avg_price_sqf', 'rooms_avg_price', 'OverallCond', 
                         'age_avg_price', 'ExterQual', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                         '2ndFlrSF', 'BsmtFinSF1', 'nbd_avg_house_price', 'BsmtFinType2', 'FullBath', 
                         'HeatingQC', 'CentralAir', '1stFlrSF', 'BsmtFullBath', 'KitchenQual', 
                         'Fireplaces', 'age_afterRemodel_price', 'SubClass_avg_price_sqf', 'Zoning_avg_price_sqf', 
                         'overalcond_avg_price', 'total_bathrooms', 'BedroomAbvGr', 'TotRmsAbvGrd', 
                        'GarageArea']


#### Full preprocessing pipeline

In [15]:
#polynomial features
poly_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
    ('variance_filter', VarianceThreshold(threshold=0.0)),  # Removes constant columns
    ('select', SelectKBest(score_func = f_regression, k = 25))
])

# Preprocessing for numeric features
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features
cat_processor = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown ='ignore', sparse_output=False))
])

#column transformer
preprocessor = ColumnTransformer(
    transformers = [
        ('poly', poly_pipeline, Selected_poly_features),
        ('num', num_pipeline, num_features),
        ('cat', cat_processor, cat_features)
    ])

# feature selector using a model (e.g., Ridge with L2 regularization)
#feature_selector = SelectFromModel(RidgeCV(alphas=np.logspace(-3, 3, 7)))

## Define the Models

In [17]:
# Base regressors
xgb = XGBRegressor(n_estimators = 500, learning_rate = 0.058, max_depth = 3, 
                   random_state = 42, verbosity = 0, n_jobs = 4)

lgbm = LGBMRegressor(n_estimators = 500, learning_rate = 0.046, max_depth = 3, 
                     random_state = 42, verbose = 0, n_jobs = 4)

rf = RandomForestRegressor(n_estimators = 500, random_state = 42, n_jobs = 4)


ridge = RidgeCV(alphas = np.logspace(-3, 3, 7))
catboost = CatBoostRegressor(verbose = 0, learning_rate = 0.03, depth = 10, random_state = 42)

base_models = [('xgb', xgb), 
               ('rf', rf),
               ('lgbm', lgbm),
               ('catboost', catboost)
              ]

#meta_model = CatBoostRegressor(verbose = 0, learning_rate = 0.038, depth = 10, random_state = 42)
meta_model = LinearRegression()


In [18]:
# stacking 
stacked_models = StackingRegressor(
    estimators = base_models,
    final_estimator = meta_model,
    cv = 5,  # Enables OOF prediction
    passthrough = False,  # Optional: If True, original features are passed to meta-model
    n_jobs = -1
)

## End-to-End ML Full Pipeline

In [20]:
# full pipeline
full_pipeline = Pipeline(steps=[
    ('feature_creator', feature_creator),  # custom feature engineer
    ('preprocessor', preprocessor),
    #('feature_selection', feature_selector), 
    ('model', stacked_models)
])

## Fit and Predict

In [22]:
full_pipeline.fit(X_train, y_train)

y_pred = full_pipeline.predict(X_test)


## Model Evaluation

In [24]:
# Metrics
rmse = np.sqrt(MSE(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE on Test Set: {rmse:.2f}")
print(f"R² on Test Set: {r2:.4f}")
print(f"MAE on Test Set: {mae:.2f}")


RMSE on Test Set: 26343.20
R² on Test Set: 0.9095
MAE on Test Set: 15337.21
